In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [9]:
#assign dates to dataframe index
NFLX_df = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_normalised.csv", index_col=0)
dates = NFLX_df.index
NFLX_df['Date'] =dates


In [10]:
NFLX_df.head()

,Open,High,Low,Close,Volume,RSI,MACD,Ichimoku_Span_A,Ichimoku_Span_B,ATR,BB_High_Indicator,BB_Low_Indicator,Date
2019-08-15,0.418144,0.415084,0.408756,0.409523,0.088711,0.459846,0.249246,0.200136,0.21118,0.111501,0.0,0.0,2019-08-15
2019-08-14,0.430238,0.426036,0.423199,0.418414,0.115910,1.000000,0.249924,0.200136,0.21118,0.125034,0.0,0.0,2019-08-14
2019-08-13,0.432740,0.437325,0.437859,0.437165,0.083349,1.000000,0.252519,0.200136,0.21118,0.140135,0.0,0.0,2019-08-13
2019-08-12,0.426614,0.432342,0.430756,0.435100,0.102925,0.910768,0.252393,0.200136,0.21118,0.143200,0.0,0.0,2019-08-12
2019-08-09,0.438382,0.437621,0.434279,0.432395,0.084289,0.800739,0.251329,0.200136,0.21118,0.147918,0.0,0.0,2019-08-09


In [4]:
# NFLX_2004 = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/2004_NFLX.csv", index_col=0)
# NFLX_2013 = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/2013_NFLX.csv", index_col=0)


In [12]:
NFLX_monthly =pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_monthly.csv", index_col=0)
NFLX_monthly.shape

(207, 5)

In [ ]:
NFLX_monthly.index = pd.to_datetime(NFLX_monthly.index, format ="%Y-%m-%d")
NFLX_monthly['Date'] = NFLX_monthly.index

In [15]:
NFLX_2015 = NFLX_monthly.iloc[NFLX_monthly.index > '2014-12-31']

In [20]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
NFLX_2015_labels = labelling(NFLX_2015)



In [21]:
NFLX_2015_flattened= get_ndays_monthly_flattened(NFLX_2015,NFLX_df,30)
# NFLX_2013_flattened= get_ndays_monthly_flattened(NFLX_2013,NFLX_df,30)



In [22]:
NFLX_2015_labels_df = pd.Series(NFLX_2015_labels).to_frame("Label")
# NFLX_2013_labels_df = pd.Series(NFLX_2013_labels).to_frame("Label")


In [23]:

#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
NFLX_2015_flat_df = pd.DataFrame.from_dict(NFLX_2015_flattened).T
# NFLX_2013_flat_df = pd.DataFrame.from_dict(NFLX_2013_flattened).T



In [24]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
NFLX_2015_flat_df = NFLX_2015_flat_df.merge(NFLX_2015_labels_df, left_index=True, right_index=True)

# NFLX_2013_flat_df  = NFLX_2013_flat_df.merge(NFLX_2013_labels_df, left_index=True, right_index=True)



In [32]:
NFLX_2015_flat_df.head()

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
2019-07-31,0.430238,0.426036,0.423199,0.418414,0.115910,1.000000,0.249924,0.200136,0.211180,0.125034,...,0.536027,0.059864,0.772802,0.280154,0.449646,0.469981,0.219864,0.0,0.0,1
2019-06-28,0.460354,0.455934,0.459597,0.456598,0.095008,0.744315,0.259892,0.200136,0.211180,0.172472,...,0.501004,0.085541,0.325860,0.216395,0.471042,0.485759,0.197345,0.0,0.0,1
2019-05-31,0.508677,0.513929,0.517315,0.519370,0.065214,0.522492,0.240427,0.449646,0.482062,0.203533,...,0.504122,0.101503,0.546746,0.256692,0.533999,0.524374,0.214815,0.0,0.0,0
2019-04-30,0.490698,0.490505,0.495805,0.493501,0.063158,0.461364,0.234093,0.517165,0.524374,0.228428,...,0.497615,0.284491,0.288726,0.228421,0.523457,0.538595,0.255085,0.0,0.0,1
2019-03-29,0.523571,0.519172,0.525854,0.521946,0.060221,0.516741,0.265624,0.531456,0.531411,0.213650,...,0.510002,0.113372,0.446663,0.252331,0.552357,0.561008,0.251341,0.0,0.0,0


In [25]:
y_test_1 = NFLX_2015_flat_df.Label

# X_test_1 = NFLX_2004_flat_df.reset_index(drop=True)
X_test_1 = NFLX_2015_flat_df.reset_index(drop=True)

In [26]:
X_test_1 = X_test_1.drop("Label", axis=1)

In [27]:
y_test_df = pd.DataFrame(y_test_1)


In [29]:
# y_test_2 = NFLX_2013_flat_df.Label

# X_test_2 = NFLX_2013_flat_df.reset_index(drop=True)

# X_test_2 = X_test_2.drop("Label", axis=1)




In [30]:
# y_test_2_df = pd.DataFrame(y_test_2)

In [28]:
X_test_1.head(1)

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,0.430238,0.426036,0.423199,0.418414,0.11591,1.0,0.249924,0.200136,0.21118,0.125034,...,0.535562,0.536027,0.059864,0.772802,0.280154,0.449646,0.469981,0.219864,0.0,0.0


In [29]:
y_test_df.head(1)

,Label
2019-07-31,1


In [33]:
# X_test_2.head(1)

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,0.239053,0.238946,0.227815,0.231324,0.108255,0.271138,0.227417,0.267279,0.312847,0.184384,...,0.125969,0.127394,0.048772,0.399884,0.284062,0.245933,0.222107,0.110292,0.0,0.0


In [34]:
# y_test_2_df.head(1)

,Label
2013-02-28,1


In [31]:
export_Xtest = X_test_1.to_csv("/Users/fahad/project_repo/data/final/NFLX/2015_NFLX_X_test.csv")
export_y_test = y_test_df.to_csv("/Users/fahad/project_repo/data/final/NFLX/2015_NFLX_y_test.csv")
# export_X_test_2 = X_test_2.to_csv("/Users/fahad/project_repo/data/final/NFLX/2013_NFLX_X_test.csv")
# export_y_test_2 = y_test_2_df.to_csv("/Users/fahad/project_repo/data/final/NFLX/2013_NFLX_y_test.csv")


In [18]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [19]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    